# ITERABLES AND ITERATORS

1. `iterable`: a collection of objects that we can iterate over and request next object until exhausted
    1. It doesnt know anything about how to iterate.
    2. It knows how to create and give us an `iterator` when we need it.
    3. implement a `__iter__()` method that returns a new iterator.  
2. `iterator`: something that is able to
    1. give us the next object when requested. It implements `__next__()` method for this.
    2. Keep track of whats given so same element is not given twice
    3. inform us when there's nothing left for it to give. It does this by raising `StopIteration` exception when next() is called if there's nothing left.
    4. will get exhausted. 

In [1]:
l = [1, 2, 3, 4, 5]

iterator = iter(l)
while True:
    try:
        print(next(iterator))
    except StopIteration:
        print("Done")
        break

print("#" * 10)
print(type(l), type(iterator))

1
2
3
4
5
Done
##########
<class 'list'> <class 'list_iterator'>


In [2]:
id(iterator)

4509317344

In [3]:
# iterator is exhausted
next(iterator)

StopIteration: 

In [4]:
# new iterator created - different id and now has values.
iterator = iter(l)
id(iterator), next(iterator)

(4510822272, 1)

In [5]:
# iterator remembers its state
next(iterator)

2

In [6]:
# now gives only remaining elements as fist 3 has been flushed
list(iterator)

[3, 4, 5]

## Filterting and Sorting iterables

In [7]:
l = [1, 3, -5, 7, -9, 11]

[el for el in l if el > 0]

[1, 3, 7, 11]

In [8]:
l = [1, 3, -5, 7, -9, 11]

def is_positive(x):
    return x > 0

[el for el in l if is_positive(el)]

[1, 3, 7, 11]

In [9]:
l = [1, 3, -5, 7, -9, 11]

# any function that returns T/F is called a predicate function
def is_positive(x):
    return x > 0

# filter takes a predicate function and an iterable and returns a lazy_iterator. 
filtered_l = filter(is_positive, l)

type(filtered_l)

filter

In [10]:
while True:
    try:
        print(next(filtered_l))
    except StopIteration:
        print('\nEnd of iterator')
        break

1
3
7
11

End of iterator


In [11]:
# we can pass in a lambda function as predicate function.
filtered_l = filter(lambda x: x%2 == 0, (2, 3, 4, 5, 6, 7))
list(filtered_l)

[2, 4, 6]

In [12]:
data = [
    {'date': '2020-04-09', 'symbol': 'APPL', 'open': 100, 'close': 250, 'high': 120},
    {'date': '2020-04-09', 'symbol': 'MSFT', 'open': 200, 'close': 320, 'high': 920},
    {'date': '2020-04-09', 'symbol': 'AMZN', 'open': 300, 'close': 410, 'high': 720},
    {'date': '2020-04-09', 'symbol': 'META', 'open': 400, 'close': 500, 'high': 420}
]

In [13]:
sorted(data, key=lambda x: x['open'])

[{'date': '2020-04-09',
  'symbol': 'APPL',
  'open': 100,
  'close': 250,
  'high': 120},
 {'date': '2020-04-09',
  'symbol': 'MSFT',
  'open': 200,
  'close': 320,
  'high': 920},
 {'date': '2020-04-09',
  'symbol': 'AMZN',
  'open': 300,
  'close': 410,
  'high': 720},
 {'date': '2020-04-09',
  'symbol': 'META',
  'open': 400,
  'close': 500,
  'high': 420}]

In [14]:
sorted(data, key=lambda x: x['high'] - x['close'])

[{'date': '2020-04-09',
  'symbol': 'APPL',
  'open': 100,
  'close': 250,
  'high': 120},
 {'date': '2020-04-09',
  'symbol': 'META',
  'open': 400,
  'close': 500,
  'high': 420},
 {'date': '2020-04-09',
  'symbol': 'AMZN',
  'open': 300,
  'close': 410,
  'high': 720},
 {'date': '2020-04-09',
  'symbol': 'MSFT',
  'open': 200,
  'close': 320,
  'high': 920}]

In [15]:
max(data, key=lambda x: x['high'])

{'date': '2020-04-09',
 'symbol': 'MSFT',
 'open': 200,
 'close': 320,
 'high': 920}

### 🧩 1. How does `list(f)` know to call `__getitem__`?

When you do:

```python
f = Fib(10)
list(f)
```

Python checks whether `f` is **iterable** — that is, whether it can be looped over.

It tries the following, in order:

1. **Check for an `__iter__` method.**

   * If present, it calls `iter(f)`, which should return an *iterator* — an object with a `__next__` method.
   * Then, it repeatedly calls `next(iterator)` until `StopIteration` is raised.

2. **If there’s no `__iter__`, fall back to the old-style sequence protocol**:

   * Python will repeatedly call `f[0]`, `f[1]`, `f[2]`, …
   * until you raise an `IndexError`.

So in your case:

* You haven’t defined `__iter__`.
* Python therefore uses the old sequence protocol.
* It calls `f[0]`, then `f[1]`, then `f[2]`, … until it hits `IndexError`.

That’s why defining `__getitem__` is enough to make your object iterable.

---

### 🔄 2. Difference between `__getitem__` and `__next__`

| Aspect                 | `__getitem__`                                                 | `__next__`                                             |
| :--------------------- | :------------------------------------------------------------ | :----------------------------------------------------- |
| Belongs to             | **Sequence protocol**                                         | **Iterator protocol**                                  |
| Usage                  | Defines how to access elements by index (`f[i]`)              | Defines how to get the *next* element in a stream      |
| Iteration style        | Python loops calling `obj[0]`, `obj[1]`, … until `IndexError` | Python loops calling `next(obj)` until `StopIteration` |
| Needs companion method | Usually used with `__len__` (optional)                        | Must have `__iter__` that returns `self`               |
| Typical use case       | For “indexable” containers (like lists, tuples, strings)      | For generators and streaming data                      |

Example of an **iterator-style Fib**:

```python
class FibIter:
    def __init__(self, n):
        self.n = n
        self.a, self.b = 0, 1
        self.count = 0

    def __iter__(self):
        return self  # iterator returns itself

    def __next__(self):
        if self.count >= self.n:
            raise StopIteration
        self.a, self.b = self.b, self.a + self.b
        self.count += 1
        return self.a
```

Now:

```python
for x in FibIter(5):
    print(x)
```

will call `__next__()` repeatedly.

---

### 🧠 Summary

* `__getitem__` → sequence-style iteration (old-school, index-based).
* `__next__` + `__iter__` → iterator-style iteration (modern, preferred).
* Python automatically falls back to `__getitem__` if `__iter__` is missing.

---

Would you like me to show you how to combine both — so your `Fib` class works with both indexing **and** `for x in f:` iteration (using `__iter__`)?


In [17]:
from functools import lru_cache

In [20]:
class Fib:
    def __init__(self, n):
        self.n = n

    def __len__(self):
        return self.n

    def __getitem__(self, s):
        if isinstance(s, int):
            if s < 0:
                s = self.n + s + 1
            if s < 0 or s > self.n:
                raise IndexError
            else:
                return Fib._fib(s)
        else:
            start, stop, step = s.indices(self.n + 1)
            rng = range(start, stop, step)
            return [Fib._fib(i) for i in rng]

    @staticmethod
    @lru_cache
    def _fib(n):
        if n < 2:
            return 1
        else:
            return Fib._fib(n-1) + Fib._fib(n-2)

#######
f = Fib(10)
f[10], f[-1]

(89, 89)

In [21]:
list(f)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]

# lets create an interable without having to have a notion of seqeunce

In [30]:
class Squares:
    def __init__(self, len_):
        self._len = len_
        self.i = 0

    def __len__(self):
        return self._len

    def next_(self):
        while self.i < self._len:
            self.i+=1
            return self.i**2
        raise StopIteration

In [31]:
s = Squares(5)
s.next_(), s.next_(), s.next_(), s.next_(), s.next_()

(1, 4, 9, 16, 25)

In [32]:
s.next_()

StopIteration: 

In [33]:
s = Squares(5)
while True:
    try:
        print(s.next_())
    except StopIteration:
        break

1
4
9
16
25


In [34]:
list(s)

TypeError: 'Squares' object is not iterable

# lets first fix our method such that we can directly call next on s instead of s.next_

In [35]:
class Squares:
    def __init__(self, len_):
        self._len = len_
        self.i = 0

    def __len__(self):
        return self._len

    def __next__(self):
        while self.i < self._len:
            self.i+=1
            return self.i**2
        raise StopIteration

In [37]:
s = Squares(5)
while True:
    try:
        print(next(s))
    except StopIteration:
        break

1
4
9
16
25


In [38]:
s.__next__()

StopIteration: 

In [39]:
s = Squares(5)
for sq in s:
    print(sq)
    

TypeError: 'Squares' object is not iterable

# lets make it a true iterable
1. Iterator protocol needs two methods.
2. `__iter__` : returns the object (class instance) iteself.
3. `__next__`: hands back the next element and raises StopIteration exception

In [40]:
class Squares:
    def __init__(self, len_):
        self._len = len_
        self.i = 0

    def __len__(self):
        return self._len

    def __next__(self):
        while self.i < self._len:
            self.i+=1
            return self.i**2
        raise StopIteration

    def __iter__(self):
        return self

In [41]:
s = Squares(5)
for sq in s:
    print(sq)

1
4
9
16
25


In [45]:
s = Squares(5)
[item for item in s]

[1, 4, 9, 16, 25]

In [47]:
s = Squares(5)
list(enumerate(s))

[(0, 1), (1, 4), (2, 9), (3, 16), (4, 25)]

In [48]:
s = Squares(5)
s[0]

TypeError: 'Squares' object is not subscriptable

# there no order in this collection hence no notion of index

In [49]:
sorted(s, reverse=True)

[25, 16, 9, 4, 1]

# lets see whats happening

In [50]:
class Squares:
    def __init__(self, len_):
        self._len = len_
        self.i = 0

    def __len__(self):
        return self._len

    def __next__(self):
        print("__next__called")
        while self.i < self._len:
            self.i+=1
            return self.i**2
        raise StopIteration

    def __iter__(self):
        print("__iter__called")
        return self

In [51]:
s = Squares(5)
while True:
    try:
        print(next(s))
    except StopIteration:
        break

__next__called
1
__next__called
4
__next__called
9
__next__called
16
__next__called
25
__next__called


In [52]:
s = Squares(5)
for item in s:
    print(item)

__iter__called
__next__called
1
__next__called
4
__next__called
9
__next__called
16
__next__called
25
__next__called


# why is the __iter__method called

In [53]:
sq = Squares(5)
sq_iterator = iter(sq)

print(hex(id(sq)), hex(id(sq_iterator)))

__iter__called
0x10d29d310 0x10d29d310


In [56]:
next(sq_iterator)

__next__called


1

In [57]:
next(sq_iterator)

__next__called


4

In [58]:
next(sq_iterator)

__next__called


9

In [59]:
next(sq_iterator)

__next__called


16

In [60]:
next(sq_iterator)

__next__called


25

In [61]:
next(sq_iterator)

__next__called


StopIteration: 

###  a collection is a bunch of data, and 2 methods which help iterate over it, each time one wants to iterate, data need not be created again, so collection is iterable but iterator is responsible for iterating over the collection and is throw away. 

In [62]:
s = Squares(5)
for item in s:
    print(item)

__iter__called
__next__called
1
__next__called
4
__next__called
9
__next__called
16
__next__called
25
__next__called


# we have exhausted s, we cant use it again without re-intantiating square class again !

In [64]:
for item in s:
    print(item)

__iter__called
__next__called


# lets seperate the data and iterator so we dont have to intialize s everytime

In [76]:
class Squares:
    def __init__(self, len_):
        self._len = len_
        self.i = 0

    def __len__(self):
        return self._len

    def __iter__(self):
        print("__iter__called")
        return self.iterSquares(self)

    class iterSquares:
        def __init__(self, squares_obj):
            self._squares_obj = squares_obj
            self.i = 0 

        def __iter__(self):
            print("sqaure obj __iter__called")
            return self
        
        def __next__(self):
            print("sqaure obj __next__called")
            while self.i < self._squares_obj._len:
                self.i+=1
                return self.i**2
            raise StopIteration

    

In [77]:
s = Squares(5)
for item in s:
    print(item)

__iter__called
sqaure obj __next__called
1
sqaure obj __next__called
4
sqaure obj __next__called
9
sqaure obj __next__called
16
sqaure obj __next__called
25
sqaure obj __next__called


In [78]:
for item in s:
    print(item)

__iter__called
sqaure obj __next__called
1
sqaure obj __next__called
4
sqaure obj __next__called
9
sqaure obj __next__called
16
sqaure obj __next__called
25
sqaure obj __next__called


In [79]:
[item for item in s]

__iter__called
sqaure obj __iter__called
sqaure obj __next__called
sqaure obj __next__called
sqaure obj __next__called
sqaure obj __next__called
sqaure obj __next__called
sqaure obj __next__called


[1, 4, 9, 16, 25]

# now we dont have to instantiate s everytime !

In [80]:
s = "i am king of the jungle"
iter_s = iter(s)
hex(id(iter_s)), hex(id(s))

('0x10daf0730', '0x10da42070')

In [81]:
next(s)

TypeError: 'str' object is not an iterator

In [82]:
next(iter_s)

'i'

# cyclical iterators

In [87]:
l = list(range(5))
l

[0, 1, 2, 3, 4]

In [98]:
class MyIterator:
    def __init__(self, list_obj):
        self._l = list_obj
        self._i = 0
        
    def __len__(self):
        return len(self._l)

    def __iter__(self):
        return self

    def __next__(self):
        result = self._l[self._i % len(self)]
        self._i += 1
        return result

In [99]:
iter_l = MyIterator(l)

In [101]:
for i in range(10):
    print(next(iter_l), end = ' ')

0 1 2 3 4 0 1 2 3 4 

### we can write our own iterators for any existing collection, even if the collection has its own implementation. 

In [124]:
suits = ['Heart', 'Spade', 'Diamond', 'Club']
faces = list(range(2, 11)) + ['J', 'Q', 'K', 'A'] 

suits, faces

(['Heart', 'Spade', 'Diamond', 'Club'],
 [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A'])

In [125]:
# lets generate a deck

class MyIterator:
    def __init__(self, list_obj, max_cycles):
        self._l = list_obj
        self._i = 0
        self._cycle = max_cycles
        
    def __len__(self):
        return len(self._l)

    def __iter__(self):
        return self

    def __next__(self):
        if self._i < self._cycle:
            result = self._l[self._i % len(self)]
            self._i += 1
            return result
        else:
            raise StopIteration
        
iter_suits = MyIterator(suits, 52)
iter_faces = MyIterator(faces, 52)
deck = list(zip(iter_suits, iter_faces))

In [126]:
len(deck), deck

(52,
 [('Heart', 2),
  ('Spade', 3),
  ('Diamond', 4),
  ('Club', 5),
  ('Heart', 6),
  ('Spade', 7),
  ('Diamond', 8),
  ('Club', 9),
  ('Heart', 10),
  ('Spade', 'J'),
  ('Diamond', 'Q'),
  ('Club', 'K'),
  ('Heart', 'A'),
  ('Spade', 2),
  ('Diamond', 3),
  ('Club', 4),
  ('Heart', 5),
  ('Spade', 6),
  ('Diamond', 7),
  ('Club', 8),
  ('Heart', 9),
  ('Spade', 10),
  ('Diamond', 'J'),
  ('Club', 'Q'),
  ('Heart', 'K'),
  ('Spade', 'A'),
  ('Diamond', 2),
  ('Club', 3),
  ('Heart', 4),
  ('Spade', 5),
  ('Diamond', 6),
  ('Club', 7),
  ('Heart', 8),
  ('Spade', 9),
  ('Diamond', 10),
  ('Club', 'J'),
  ('Heart', 'Q'),
  ('Spade', 'K'),
  ('Diamond', 'A'),
  ('Club', 2),
  ('Heart', 3),
  ('Spade', 4),
  ('Diamond', 5),
  ('Club', 6),
  ('Heart', 7),
  ('Spade', 8),
  ('Diamond', 9),
  ('Club', 10),
  ('Heart', 'J'),
  ('Spade', 'Q'),
  ('Diamond', 'K'),
  ('Club', 'A')])

In [127]:
# better way 

# Make 52 cards (13 faces × 4 suits)
deck = [(face, suit) for suit in suits for face in faces]

# Optional: to use your iterator class
iter_deck = MyIterator(deck, len(deck))
print(list(iter_deck))

[(2, 'Heart'), (3, 'Heart'), (4, 'Heart'), (5, 'Heart'), (6, 'Heart'), (7, 'Heart'), (8, 'Heart'), (9, 'Heart'), (10, 'Heart'), ('J', 'Heart'), ('Q', 'Heart'), ('K', 'Heart'), ('A', 'Heart'), (2, 'Spade'), (3, 'Spade'), (4, 'Spade'), (5, 'Spade'), (6, 'Spade'), (7, 'Spade'), (8, 'Spade'), (9, 'Spade'), (10, 'Spade'), ('J', 'Spade'), ('Q', 'Spade'), ('K', 'Spade'), ('A', 'Spade'), (2, 'Diamond'), (3, 'Diamond'), (4, 'Diamond'), (5, 'Diamond'), (6, 'Diamond'), (7, 'Diamond'), (8, 'Diamond'), (9, 'Diamond'), (10, 'Diamond'), ('J', 'Diamond'), ('Q', 'Diamond'), ('K', 'Diamond'), ('A', 'Diamond'), (2, 'Club'), (3, 'Club'), (4, 'Club'), (5, 'Club'), (6, 'Club'), (7, 'Club'), (8, 'Club'), (9, 'Club'), (10, 'Club'), ('J', 'Club'), ('Q', 'Club'), ('K', 'Club'), ('A', 'Club')]


# full deck of cards as iterator

In [136]:
import random

class Deck:
    def __init__(self):
        suits = ['Hearts', 'Spades', 'Diamonds', 'Clubs']
        faces = list(range(2, 11)) + ['J', 'Q', 'K', 'A']
        # Create 52 cards as tuples (face, suit)
        self._cards = [(f, s) for s in suits for f in faces]
        self._i = 0  # internal index

    def shuffle(self):
        random.shuffle(self._cards)
        self._i = 0  # reset index after shuffling

    def __len__(self):
        return len(self._cards)

    def __iter__(self):
        return self

    def __next__(self):
        if self._i >= len(self._cards):
            raise StopIteration
        card = self._cards[self._i]
        self._i += 1
        return card

    def draw(self, n=1):
        """Draw n cards from the top of the deck."""
        if self._i + n > len(self._cards):
            raise ValueError("Not enough cards left to draw.")
        cards = self._cards[self._i : self._i + n]
        self._i += n
        return cards

    def __repr__(self):
        return f"<Deck of {len(self._cards) - self._i} cards remaining>"


In [139]:
deck = Deck()
print("Before shuffle:", deck._cards[:5])  # peek at first few cards
print(deck)

deck.shuffle()
print("\nAfter shuffle:", deck._cards[:5])

# Draw some cards
print("\nDrawing 3 cards:", deck.draw(3))
print(deck)
# You can also iterate directly
print("\nIterating:")
for card in deck:
    print(card)

Before shuffle: [(2, 'Hearts'), (3, 'Hearts'), (4, 'Hearts'), (5, 'Hearts'), (6, 'Hearts')]
<Deck of 52 cards remaining>

After shuffle: [(10, 'Hearts'), (6, 'Diamonds'), (6, 'Hearts'), ('K', 'Spades'), (6, 'Clubs')]

Drawing 3 cards: [(10, 'Hearts'), (6, 'Diamonds'), (6, 'Hearts')]
<Deck of 49 cards remaining>

Iterating:
('K', 'Spades')
(6, 'Clubs')
('J', 'Diamonds')
(8, 'Diamonds')
('J', 'Spades')
(2, 'Spades')
('Q', 'Clubs')
(10, 'Clubs')
(10, 'Spades')
(4, 'Diamonds')
(9, 'Hearts')
('K', 'Diamonds')
(5, 'Spades')
(7, 'Spades')
(7, 'Hearts')
(5, 'Hearts')
('K', 'Clubs')
(8, 'Hearts')
('Q', 'Spades')
('A', 'Diamonds')
(3, 'Clubs')
(9, 'Spades')
(9, 'Clubs')
('K', 'Hearts')
(2, 'Diamonds')
(2, 'Clubs')
(3, 'Spades')
('J', 'Hearts')
(7, 'Clubs')
('Q', 'Diamonds')
(6, 'Spades')
('A', 'Spades')
(3, 'Hearts')
(4, 'Hearts')
(10, 'Diamonds')
(2, 'Hearts')
(7, 'Diamonds')
(9, 'Diamonds')
(8, 'Clubs')
(5, 'Diamonds')
('A', 'Hearts')
('Q', 'Hearts')
(4, 'Clubs')
(8, 'Spades')
(4, 'Spades')
(5

# using `itertools`

In [145]:
class MyIterator:
    def __init__(self, list_obj):
        self._l = list_obj
        self._i = 0
        
    def __len__(self):
        return len(self._l)

    def __iter__(self):
        return self

    def __next__(self):
        result = self._l[self._i % len(self)]
        self._i += 1
        return result


n = 10
iter_cycle = MyIterator('NSWE')
items = [f'{i}{next(iter_cycle)}' for i in range(1, n+1)]

items

['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']

In [146]:
import itertools

help(itertools.cycle)

Help on class cycle in module itertools:

class cycle(builtins.object)
 |  cycle(iterable, /)
 |
 |  Return elements from the iterable until it is exhausted. Then repeat the sequence indefinitely.
 |
 |  Methods defined here:
 |
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |
 |  __iter__(self, /)
 |      Implement iter(self).
 |
 |  __next__(self, /)
 |      Implement next(self).
 |
 |  __reduce__(self, /)
 |      Return state information for pickling.
 |
 |  __setstate__(self, object, /)
 |      Set state information for unpickling.
 |
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |
 |  __new__(*args, **kwargs)
 |      Create and return a new object.  See help(type) for accurate signature.



In [147]:
n = 10
iter_cycle = itertools.cycle('NSWE')
items = [f'{i}{next(iter_cycle)}' for i in range(1, n+1)]

items

['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']

# Lets implement a deck of cards using this itertools.cycle

In [148]:
suits = ['Hearts', 'Spades', 'Diamonds', 'Clubs']
faces = list(range(2, 11)) + ['J', 'Q', 'K', 'A']

suits, faces

(['Hearts', 'Spades', 'Diamonds', 'Clubs'],
 [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A'])

In [153]:
iter_faces = itertools.cycle(faces)

[f'{suit} {next(iter_faces)}' for suit in suits for face in faces]

['Hearts 2',
 'Hearts 3',
 'Hearts 4',
 'Hearts 5',
 'Hearts 6',
 'Hearts 7',
 'Hearts 8',
 'Hearts 9',
 'Hearts 10',
 'Hearts J',
 'Hearts Q',
 'Hearts K',
 'Hearts A',
 'Spades 2',
 'Spades 3',
 'Spades 4',
 'Spades 5',
 'Spades 6',
 'Spades 7',
 'Spades 8',
 'Spades 9',
 'Spades 10',
 'Spades J',
 'Spades Q',
 'Spades K',
 'Spades A',
 'Diamonds 2',
 'Diamonds 3',
 'Diamonds 4',
 'Diamonds 5',
 'Diamonds 6',
 'Diamonds 7',
 'Diamonds 8',
 'Diamonds 9',
 'Diamonds 10',
 'Diamonds J',
 'Diamonds Q',
 'Diamonds K',
 'Diamonds A',
 'Clubs 2',
 'Clubs 3',
 'Clubs 4',
 'Clubs 5',
 'Clubs 6',
 'Clubs 7',
 'Clubs 8',
 'Clubs 9',
 'Clubs 10',
 'Clubs J',
 'Clubs Q',
 'Clubs K',
 'Clubs A']

# cyclic iterator for unordered collections

In [155]:
l = set('abcd')
l

{'a', 'b', 'c', 'd'}

### we cant use any index with unordered collections

In [158]:
len(l)

4

In [159]:
l[0]

TypeError: 'set' object is not subscriptable

In [160]:
class MyIterator:
    def __init__(self, obj):
        self._obj = obj
        self._iterable = iter(self._obj)
        
    def __iter__(self):
        return self

    def __next__(self):
        return next(self._iterable)
        

In [162]:
iter_l = MyIterator(l)

In [164]:
for i in range(10):
    print(i, next(iter_l))

0 d
1 a
2 c
3 b


StopIteration: 

In [167]:
class MyCyclicIterator:
    def __init__(self, obj):
        self._obj = obj
        self._iterable = iter(self._obj)
        
    def __iter__(self):
        return self

    def __next__(self):
        try:
            result = next(self._iterable)
        except StopIteration:
            self._iterable = iter(self._obj)
            result = next(self._iterable)
        return result

In [168]:
iter_l = MyCyclicIterator(l)
for i in range(10):
    print(i, next(iter_l))

0 d
1 a
2 c
3 b
4 d
5 a
6 c
7 b
8 d
9 a


# lazy evaluation of iterables

In [170]:
import math

In [172]:
class Circle:
    def __init__(self, radius):
        self.radius = radius

    @property
    def radius(self):
        return self._radius

    @radius.setter
    def radius(self, radius):
        if radius <= 0:
            raise ValueError('Radius of a Cicle cant be negative')
        self._radius = radius

    def area(self):
        print('calculating area')
        return math.pi * (self.radius ** 2)

    def __repr__(self):
        return f'Circle(r={self.radius})'
        

In [180]:
c1 = Circle(10)
c1

Circle(r=10)

In [181]:
c1.radius

10

In [182]:
c1.area()

calculating area


314.1592653589793

# change radius and call area again - area updates 

In [184]:
c1.radius = 2
c1.area()

calculating area


12.566370614359172

### ..but area is calculated even if radius did not change, what if our program creates a circle once but calls area many times - why calculate it each time !!!

In [185]:
c1.area()

calculating area


12.566370614359172

In [186]:
c1.area()

calculating area


12.566370614359172

## lets fix it with lazy evaluation

In [188]:
class Circle:
    def __init__(self, radius):
        self.radius = radius
        self._area = None

    @property
    def radius(self):
        return self._radius

    @radius.setter
    def radius(self, radius):
        if radius <= 0:
            raise ValueError('Radius of a Cicle cant be negative')
        self._radius = radius

    @property
    def area(self):
        if self._area is None:
            print('calculating area')
            self._area = math.pi * (self.radius ** 2)
        return self._area

    def __repr__(self):
        return f'Circle(r={self.radius})'

In [189]:
c1 = Circle(10)
c1.area

calculating area


314.1592653589793

In [190]:
c1.area

314.1592653589793

### area is not calculated again but... now if we change radius then old area is returned. We somehow have to change area when radius changes -- but why calcuate it without it being asked ! how can we fix this ?

In [192]:
c1.radius = 2
c1.area

314.1592653589793

In [193]:
class Circle:
    def __init__(self, radius):
        self.radius = radius
        self._area = None

    @property
    def radius(self):
        return self._radius

    @radius.setter
    def radius(self, radius):
        if radius <= 0:
            raise ValueError('Radius of a Cicle cant be negative')
        self._radius = radius
        self._area = None  # we just invalidate the area each time radius changes

    @property
    def area(self):
        if self._area is None:
            print('calculating area')
            self._area = math.pi * (self.radius ** 2)
        return self._area

    def __repr__(self):
        return f'Circle(r={self.radius})'

In [194]:
c1 = Circle(10)
c1.area

calculating area


314.1592653589793

In [195]:
c1.area

314.1592653589793

In [197]:
c1.radius = 2

In [198]:
c1.area

calculating area


12.566370614359172

In [199]:
c1.area

12.566370614359172

# how to find if something is an iterable or iterator

| Concept                            | **Iterable**                                                             | **Iterator**                                                                   |
| :--------------------------------- | :----------------------------------------------------------------------- | :----------------------------------------------------------------------------- |
| Definition                         | An object that can return an **iterator** when `iter()` is called on it. | An object that knows how to **return the next value** when `next()` is called. |
| Required methods                   | Must implement `__iter__()` (returns an iterator).                       | Must implement **both** `__iter__()` and `__next__()`.                         |
| Can be looped over multiple times? | ✅ Yes (each time creates a *fresh iterator*).                            | ⚠️ No — it’s consumed after one pass.                                          |
| Example objects                    | `list`, `tuple`, `set`, `dict`, `str`, `range`                           | File objects, generators, results of `iter(...)`, or any custom iterator class |
| State                              | Stateless — no internal progress                                         | Stateful — remembers where it is in the iteration                              |


In [224]:
# iterable : doesnt exhaust

l = [1, 2, 3]

type(l), '__iter__' in dir(l), '__next__' in dir(l), iter(l) is l

(list, True, False, False)

In [225]:
# iterable : doesnt exhaust

l = set([1, 2, 3])

type(l), '__iter__' in dir(l), '__next__' in dir(l), iter(l) is l

(set, True, False, False)

In [226]:
# iterable : doesnt exhaust

l = range(5)

type(l), '__iter__' in dir(l), '__next__' in dir(l), iter(l) is l

(range, True, False, False)

In [227]:
# iterator : exhausts

l = zip([1, 2, 3], 'abc')

type(l), '__iter__' in dir(l), '__next__' in dir(l), iter(l) is l

(zip, True, True, True)

In [230]:
# iterator : exhausts

l = enumerate([1, 2, 3])

type(l), '__iter__' in dir(l), '__next__' in dir(l), iter(l) is l

(enumerate, True, True, True)

In [232]:
# iteratrable : doesnt exhaust

l = {'a':2, 'b':3}.keys()

type(l), '__iter__' in dir(l), '__next__' in dir(l), iter(l) is l

(dict_keys, True, False, False)

# iterator over callables

In [233]:
def counter():
    count = 0
    def inner():
        nonlocal count
        count += 1
        return count
    return inner

cnt = counter()

In [234]:
cnt()

1

In [235]:
cnt()

2

In [238]:
for _ in range(5):
    print(cnt())

8
9
10
11
12


# how to write an iterable on it such that we dont have to create an object everytime

In [244]:
class CounterIterator:
    def __init__(self, callabe_obj):
        self._callable = callabe_obj
        
    def __iter__(self):
        return self

    def __next__(self):
        return self._callable()

In [245]:
def counter():
    count = 0
    def inner():
        nonlocal count
        count += 1
        return count
    return inner

cnt = counter()

cnt_iter = CounterIterator(cnt)

for _ in range(5):
    print(next(cnt_iter))

1
2
3
4
5


### just an experiment to see if we can write entire thing as a class - without having to intantiating iterator everytime

In [253]:
class Counter:
    def __init__(self):
        # self._count = 0
        pass

    def __iter__(self):
        return self.iterCounter(self)

    class iterCounter:
        def __init__(self, counter_obj):
            self._counter = counter_obj
            self._count = 0

        def __iter__(self):
            return self

        def __next__(self):
            self._count += 1
            return self._count
            
        

cnt = Counter()
i = 0
for x in cnt:
    i += 1
    if i < 5:
        print(i, x)
    else:
        break

1 1
2 2
3 3
4 4


In [255]:
[x for x, y in zip(cnt, [1,2,3,4,5,6])]

[1, 2, 3, 4, 5, 6]

# sentinel value to have finite iterables over a callable

In [14]:
def counter():
    count = 0
    def inner():
        nonlocal count
        count += 1
        return count
    return inner


# write our own iterator over exiting callables
class CounterIterator:
    def __init__(self, callabe_obj, sentinel):
        self._callable = callabe_obj
        self._sentinel = sentinel
        
    def __iter__(self):
        return self

    def __next__(self):
        result = self._callable()
        
        if result == self._sentinel:
            raise StopIteration
            
        return result

        
cnt = counter()

cnt_iter = CounterIterator(cnt, 5)

for c in cnt_iter:
    print(c)

1
2
3
4


In [9]:
next(cnt_iter)

6

# what has happened here ! 
### well we raises `StopIteration` error, but nothing stops us from calling the iterator again and this time result has moved outside sentinel value. 
#### if we do result >= self._sentinel - it wont be pythonic 

In [12]:
def counter():
    count = 0
    def inner():
        nonlocal count
        count += 1
        return count
    return inner


# write our own iterator over exiting objects
class CounterIterator:
    def __init__(self, callabe_obj, sentinel):
        self._callable = callabe_obj
        self._sentinel = sentinel
        self._done = False
        
    def __iter__(self):
        return self

    def __next__(self):
        if self._done:
            raise StopIteration
        
        result = self._callable()
        
        if result == self._sentinel:
            self._done = True
            raise StopIteration
        return result

        
cnt = counter()

cnt_iter = CounterIterator(cnt, 5)

for c in cnt_iter:
    print(c)

1
2
3
4


In [13]:
next(cnt_iter)

StopIteration: 

# now with our own callable

In [6]:
# create our own objects with iterable and iterator
class Counter:
    def __init__(self, sentinel):
        self._sentinel = sentinel

    def __iter__(self):
        return self.iterCounter(self)

    class iterCounter:
        def __init__(self, counter_obj):
            self._counter = counter_obj
            self._count = 0

        def __iter__(self):
            return self

        def __next__(self):
            self._count += 1
            result = self._count
            
            if result == self._counter._sentinel:
                raise StopIteration
            
            return result

cnt = Counter(5)
[x for x in cnt]

[1, 2, 3, 4]

In [4]:
list(cnt)

[1, 2, 3, 4]

In [7]:
next(cnt)

TypeError: 'Counter' object is not an iterator

# this is what python exactly does

In [15]:
help(iter)

Help on built-in function iter in module builtins:

iter(...)
    iter(iterable) -> iterator
    iter(callable, sentinel) -> iterator

    Get an iterator from an object.  In the first form, the argument must
    supply its own iterator, or be a sequence.
    In the second form, the callable is called until it returns the sentinel.



## lets use pythons built-in iterator

In [16]:
def my_counter():
    count = 0
    def inner():
        nonlocal count
        count += 1
        return count
    return inner

cnt = my_counter()
cnt_iter = iter(cnt, 7)

for x in cnt_iter:
    print(x)

1
2
3
4
5
6


In [17]:
next(cnt_iter)

StopIteration: 

### lets say we want to generate random numbers until we hit a number 8 

In [18]:
import random

In [19]:
random.seed(0)

for i in range(10):
    print(random.randint(0, 10))

6
6
0
4
8
7
6
4
7
5


In [20]:
rnd = iter(random.randint(0, 10), 8)

TypeError: iter(v, w): v must be callable

### Here’s what happens:

1. random.randint(0, 10) is called immediately (not passed as a callable).
2. That produces a single integer, say 7.
3. So effectively you’re doing: `rnd = iter(7, 8)`
4. and `7` is not callable

In [24]:
rnd = iter(lambda: random.randint(0, 10), 8)

#### since we have set the seed same as above - we would get same set of random numbers but now we have set 8 as sentinel value

In [25]:
random.seed(0)
for x in rnd:
    print(x)

6
6
0
4


# delegating iterators

In [26]:
import collections

In [27]:
Person = collections.namedtuple('Person', ['first', 'last'])

In [33]:
class PersonNames:
    def __init__(self, persons_list):
        self._persons = [x.first + " " + x.last for x in persons_list]

In [34]:
l = [ Person('John', 'Cena'),
      Person('Hulk', 'Hogan'),
      Person('Fred', 'Baptiste')
]

person_names = PersonNames(l)

In [35]:
person_names._persons

['John Cena', 'Hulk Hogan', 'Fred Baptiste']

In [45]:
class PersonNames:
    def __init__(self, persons_list):
        self._persons = [x.first + " " + x.last for x in persons_list]

    def __iter__(self):
        return self.PersonIterator(self)

    class PersonIterator:
        def __init__(self, PersonNames_obj):
            self._obj = PersonNames_obj
            self._counter = 0

        def __iter__(self):
            return self

        def __next__(self):
            if self._counter >= len(self._obj):
                raise StopIteration
            else:
                result = self._obj[self._counter]
                self._counter += 1
                return result



In [46]:
l = [ Person('John', 'Cena'),
      Person('Hulk', 'Hogan'),
      Person('Fred', 'Baptiste')
]

person_names = PersonNames(l)

In [47]:
for person in person_names:
    print(person)

TypeError: object of type 'PersonNames' has no len()

# lets add that

In [49]:
class PersonNames:
    def __init__(self, persons_list):
        self._persons = [x.first + " " + x.last for x in persons_list]

    def __len__(self):
        return len(self._persons)
    
    def __iter__(self):
        return self.PersonIterator(self)

    class PersonIterator:
        def __init__(self, PersonNames_obj):
            self._obj = PersonNames_obj
            self._counter = 0

        def __iter__(self):
            return self

        def __next__(self):
            if self._counter >= len(self._obj):
                raise StopIteration
            else:
                result = self._obj[self._counter]
                self._counter += 1
                return result


l = [ Person('John', 'Cena'),
      Person('Hulk', 'Hogan'),
      Person('Fred', 'Baptiste')
]

person_names = PersonNames(l)

for person in person_names:
    print(person)

TypeError: 'PersonNames' object is not subscriptable

# lets implement a `__getitem__` method

In [50]:
class PersonNames:
    def __init__(self, persons_list):
        self._persons = [x.first + " " + x.last for x in persons_list]

    def __len__(self):
        return len(self._persons)

    def __getitem__(self, s):
        return self._persons[s]
    
    def __iter__(self):
        return self.PersonIterator(self)

    class PersonIterator:
        def __init__(self, PersonNames_obj):
            self._obj = PersonNames_obj
            self._counter = 0

        def __iter__(self):
            return self

        def __next__(self):
            if self._counter >= len(self._obj):
                raise StopIteration
            else:
                result = self._obj[self._counter]
                self._counter += 1
                return result


l = [ Person('John', 'Cena'),
      Person('Hulk', 'Hogan'),
      Person('Fred', 'Baptiste')
]

person_names = PersonNames(l)

for person in person_names:
    print(person)

John Cena
Hulk Hogan
Fred Baptiste


In [51]:
list(person_names)

['John Cena', 'Hulk Hogan', 'Fred Baptiste']

### lets make it a bit more robust

In [52]:
class PersonNames:
    def __init__(self, persons_list):
        try:
            self._persons = [x.first + " " + x.last for x in persons_list]
        # if persons_list is not iterable, or if lists element doesnt have .first or .last attributes 
        except (TypeError, AttributeError):
            self._person = []

    def __len__(self):
        return len(self._persons)

    def __getitem__(self, s):
        return self._persons[s]
    
    def __iter__(self):
        return self.PersonIterator(self)

    class PersonIterator:
        def __init__(self, PersonNames_obj):
            self._obj = PersonNames_obj
            self._counter = 0

        def __iter__(self):
            return self

        def __next__(self):
            if self._counter >= len(self._obj):
                raise StopIteration
            else:
                result = self._obj[self._counter]
                self._counter += 1
                return result


l = [ Person('John', 'Cena'),
      Person('Hulk', 'Hogan'),
      Person('Fred', 'Baptiste')
]

person_names = PersonNames(l)

for person in person_names:
    print(person)

John Cena
Hulk Hogan
Fred Baptiste


## we can do better - see this

In [53]:
class PersonNames:
    def __init__(self, persons_list):
        try:
            self._persons = [x.first + " " + x.last for x in persons_list]
        # if persons_list is not iterable, or if lists element doesnt have .first or .last attributes 
        except (TypeError, AttributeError):
            self._person = []
    
    def __iter__(self):
        return iter(self._persons)


l = [ Person('John', 'Cena'),
      Person('Hulk', 'Hogan'),
      Person('Fred', 'Baptiste')
]

person_names = PersonNames(l)

for person in person_names:
    print(person)

John Cena
Hulk Hogan
Fred Baptiste


In [54]:
list(person_names)

['John Cena', 'Hulk Hogan', 'Fred Baptiste']

In [61]:
person_iterator = iter(person_names)

print(person_iterator is person_names, end='\n\n')

for _ in range(5):
    try:
        print(next(person_iterator))
    except StopIteration:
        break

False

John Cena
Hulk Hogan
Fred Baptiste


# REVERSED ITERATION

In [64]:
_SUITS = ('Spades', 'Hearts', 'Diamonds', 'Clubs')
_RANKS = tuple(range(2, 11)) + tuple('JQKA')

In [65]:
Card = collections.namedtuple('Card', ['rank', 'suit'])

In [84]:
class CardDeck:
    def __init__(self):
        self.length = len(_SUITS) * len(_RANKS)

    def __len__(self):
        return self.length

    def __iter__(self):
        return self.CardDeckIterator(self.length)

    class CardDeckIterator:
        def __init__(self, length):
            self.length = length
            self.idx = 0

        def __iter__(self):
            return self

        def __next__(self):
            if self.idx == self.length:
                raise StopIteration
            else:
                suit = _SUITS[self.idx // len(_RANKS)]
                rank = _RANKS[self.idx % len(_RANKS)]
                self.idx += 1
                return Card(rank, suit)

mydeck = CardDeck()

In [91]:
for card in mydeck:
    print(card)

Card(rank=2, suit='Spades')
Card(rank=3, suit='Spades')
Card(rank=4, suit='Spades')
Card(rank=5, suit='Spades')
Card(rank=6, suit='Spades')
Card(rank=7, suit='Spades')
Card(rank=8, suit='Spades')
Card(rank=9, suit='Spades')
Card(rank=10, suit='Spades')
Card(rank='J', suit='Spades')
Card(rank='Q', suit='Spades')
Card(rank='K', suit='Spades')
Card(rank='A', suit='Spades')
Card(rank=2, suit='Hearts')
Card(rank=3, suit='Hearts')
Card(rank=4, suit='Hearts')
Card(rank=5, suit='Hearts')
Card(rank=6, suit='Hearts')
Card(rank=7, suit='Hearts')
Card(rank=8, suit='Hearts')
Card(rank=9, suit='Hearts')
Card(rank=10, suit='Hearts')
Card(rank='J', suit='Hearts')
Card(rank='Q', suit='Hearts')
Card(rank='K', suit='Hearts')
Card(rank='A', suit='Hearts')
Card(rank=2, suit='Diamonds')
Card(rank=3, suit='Diamonds')
Card(rank=4, suit='Diamonds')
Card(rank=5, suit='Diamonds')
Card(rank=6, suit='Diamonds')
Card(rank=7, suit='Diamonds')
Card(rank=8, suit='Diamonds')
Card(rank=9, suit='Diamonds')
Card(rank=10, 

# if we want last 5 cards - without materializing all the cards

In [92]:
l = [1, 2, 3, 4]
reversed(l)

In [93]:
list(reversed(l))

[4, 3, 2, 1]

In [94]:
reversed_deck = reversed(mydeck)

TypeError: 'CardDeck' object is not reversible

# lets implement `__reversed__` to make it reversible

In [95]:
class CardDeck:
    def __init__(self):
        self.length = len(_SUITS) * len(_RANKS)

    def __len__(self):
        return self.length

    def __iter__(self):
        return self.CardDeckIterator(self.length)

    def __reversed__(self):
        return self.CardDeckIterator(self.length)

    class CardDeckIterator:
        def __init__(self, length):
            self.length = length
            self.idx = 0

        def __iter__(self):
            return self

        def __next__(self):
            if self.idx == self.length:
                raise StopIteration
            else:
                suit = _SUITS[self.idx // len(_RANKS)]
                rank = _RANKS[self.idx % len(_RANKS)]
                self.idx += 1
                return Card(rank, suit)

mydeck = CardDeck()
for card in reversed(mydeck):
    if card.rank == 10:
        break
    print(card)

Card(rank=2, suit='Spades')
Card(rank=3, suit='Spades')
Card(rank=4, suit='Spades')
Card(rank=5, suit='Spades')
Card(rank=6, suit='Spades')
Card(rank=7, suit='Spades')
Card(rank=8, suit='Spades')
Card(rank=9, suit='Spades')


# now that we know how to make it reversible lets fix the reversed method

In [100]:
class CardDeck:
    def __init__(self):
        self.length = len(_SUITS) * len(_RANKS)

    def __len__(self):
        return self.length

    def __iter__(self):
        return self.CardDeckIterator(self.length)

    def __reversed__(self):
        return self.CardDeckIterator(self.length, reverse= True)

    class CardDeckIterator:
        def __init__(self, length, reverse=False):
            self.length = length
            self.idx = 0
            self.reverse = reverse

        def __iter__(self):
            return self

        def __next__(self):
            if self.idx == self.length:
                raise StopIteration
            else:
                if self.reverse:
                    index = self.length - 1 - self.idx
                else:
                    index = self.idx
                    
                suit = _SUITS[index // len(_RANKS)]
                rank = _RANKS[index % len(_RANKS)]
                self.idx += 1
                return Card(rank, suit)

mydeck = CardDeck()

for card in reversed(mydeck):
    print(card)

Card(rank='A', suit='Clubs')
Card(rank='K', suit='Clubs')
Card(rank='Q', suit='Clubs')
Card(rank='J', suit='Clubs')
Card(rank=10, suit='Clubs')
Card(rank=9, suit='Clubs')
Card(rank=8, suit='Clubs')
Card(rank=7, suit='Clubs')
Card(rank=6, suit='Clubs')
Card(rank=5, suit='Clubs')
Card(rank=4, suit='Clubs')
Card(rank=3, suit='Clubs')
Card(rank=2, suit='Clubs')
Card(rank='A', suit='Diamonds')
Card(rank='K', suit='Diamonds')
Card(rank='Q', suit='Diamonds')
Card(rank='J', suit='Diamonds')
Card(rank=10, suit='Diamonds')
Card(rank=9, suit='Diamonds')
Card(rank=8, suit='Diamonds')
Card(rank=7, suit='Diamonds')
Card(rank=6, suit='Diamonds')
Card(rank=5, suit='Diamonds')
Card(rank=4, suit='Diamonds')
Card(rank=3, suit='Diamonds')
Card(rank=2, suit='Diamonds')
Card(rank='A', suit='Hearts')
Card(rank='K', suit='Hearts')
Card(rank='Q', suit='Hearts')
Card(rank='J', suit='Hearts')
Card(rank=10, suit='Hearts')
Card(rank=9, suit='Hearts')
Card(rank=8, suit='Hearts')
Card(rank=7, suit='Hearts')
Card(ran

# python in general is able to reverse a sequence type but not an iterable - see this

In [110]:
class Squares:
    def __init__(self, length):
        self.squares = [i**2 for i in range(length)]

    # def __len__(self):
    #     return len(self.squares)

    def __getitem__(self, s):
        return self.squares[s]

In [111]:
mysquares = Squares(5)
mysquares[3]

9

In [112]:
for num in mysquares:
    print(num)

0
1
4
9
16


In [113]:
for num in reversed(mysquares):
    print(num)

TypeError: object of type 'Squares' has no len()

In [116]:
class Squares:
    def __init__(self, length):
        self.squares = [i**2 for i in range(length)]

    def __len__(self):
        return len(self.squares)

    def __getitem__(self, s):
        return self.squares[s]

In [117]:
mysquares = Squares(5)
for num in reversed(mysquares):
    print(num)

16
9
4
1
0
